# Lab No 5: Intro to Networks

# Challenge 1

In [ ]:
import pandas as pd
import numpy as np

# Importing the csv. file.....
df = pd.read_csv('/UA/Lab_5_Data/FS.csv')

In [ ]:
import networkx as nx

# Plugging in the csv. into nx
# With column source as source and target as target
G = nx.from_pandas_edgelist(df, source="source", target="target")

# Showing how many nodes and edges in the dataset
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

In [ ]:
# Calculating the degree centrality
degree_centrality = nx.degree_centrality(G)

# Sorting out the top 4 nodes with the greatest value
top4_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:4]

print("Top 4 most important nodes:", top4_nodes)

In [ ]:
# Creating a subset to avoid plotting all the nodes
subgraph_nodes = set(top4_nodes)
for node in top4_nodes:
    
    # Since they are the most important nodes 
    # They must have a lot of neighbors, the graph will be a mess if we plot it all
    # extracting only 50 neighbors for each nodes 
    neighbors = list(G.neighbors(node))[:50]  
    subgraph_nodes.update(neighbors)

# Create a subgraph
G_sub = G.subgraph(subgraph_nodes)  

In [ ]:
import matplotlib.pyplot as plt

# Plotting the subgraph
plt.figure(figsize=(10, 10))

pos = nx.spring_layout(G_sub)  
nx.draw(G_sub, pos, 
        with_labels=True, 
        node_color='yellow', 
        edge_color='gray',
        node_size=500, 
        font_size=8)

nx.draw_networkx_nodes(G_sub, 
                       pos, 
                       nodelist=top4_nodes, 
                       node_color='red', 
                       node_size=800)


plt.title("Top 4 Most Important Nodes and Their Neighbors")
plt.show()

In [ ]:
# Creating a list for the original network degree centrality
centrality_values = list(degree_centrality.values())

# These values mostly fall around 0.0001 to 0.000001
# They would all fall in the same bar if we plot it directly
# Putting the number into log form will provide a better illustration
bins = np.logspace(np.log10(min(centrality_values) + 1e-5), np.log10(max(centrality_values)), 50)

#Plotting the graph
plt.figure(figsize=(10, 6))
plt.hist(centrality_values, bins=bins, color='blue', edgecolor='black', alpha=0.7)
plt.xscale("log")
plt.xlabel("Degree Centrality (Log Scale)")
plt.ylabel("Frequency")
plt.title("Distribution of Node Degree Centrality in the Network")

plt.show()

In [ ]:
import matplotlib.cm as color
import matplotlib.colors as colors
from sklearn.preprocessing import MinMaxScaler

# Sorting top 25 nodes out by degree
top25_nodes = sorted(G.degree, key=lambda x: x[1], reverse=True)[:25]

# Creating a new subgraph
G_small = G.subgraph([n for n, _ in top_nodes]).copy()

# Computing the betweenness centrality
centrality = nx.betweenness_centrality(G_small, endpoints=True)

In [ ]:
# Node layout
pos = nx.spring_layout(G_small, seed=42)

# Since the value of is around 0.0822 to 0.0808
# It will be insignificant if we use it to represent the node sizes in the graph directly
# So It will be better if we standardize it

# Converting the betweenness centrality values into NumPy array, for plugging into MinMaxScaler
centrality_values = np.array(list(centrality.values())).reshape(-1, 1)
scaler = MinMaxScaler()
centrality_scaled = scaler.fit_transform(centrality_values).flatten()
node_sizes = [v * 3000 for v in centrality_scaled] 

# Color by the betweenness centrality values
centrality_values = list(centrality.values())
norm = colors.Normalize(vmin=min(centrality_values), vmax=max(centrality_values))
cmap = plt.colormaps["viridis"]
node_colors = [cmap(norm(centrality[n])) for n in G_small.nodes()]

In [ ]:
# Plotting the graph
fig, ax = plt.subplots(figsize=(14, 10))
nx.draw_networkx(
    G_small,
    pos=pos,
    node_size=node_sizes,
    node_color=node_colors,
    edge_color="lightgray",
    with_labels=True,
    alpha=0.8,
    ax=ax
)

# Add a colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label("Betweenness Centrality", fontsize=12)


ax.set_title("Top 25 Nodes — Betweenness Centrality (Size + Color)", fontsize=18, fontweight="bold")
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
import nxviz as nv
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="nxviz")
nv.MatrixPlot(G_sub)
plt.show()

In [ ]:
nv.CircosPlot(G_sub)
plt.show()

In [ ]:
a = nv.ArcPlot(G_sub)
plt.show()

# Challenge 2

In [ ]:
import networkx as nx
import osmnx as ox

In [ ]:
warnings.filterwarnings('ignore')

# Downloading the street network of Kowloon, Hong Kong
place = "Kowloon, Hong Kong"
G = ox.graph_from_place(place, network_type="drive")

In [ ]:
warnings.filterwarnings('ignore')

# Calculating the geographic area covered 
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
graph_area_m = nodes_proj.unary_union.convex_hull.area
graph_area_m

In [ ]:
# Basic Statistics of the street network
ox.basic_stats(G, area=graph_area_m, clean_int_tol=15)

In [ ]:
# Customized street network graph
ox.plot.plot_graph(G, 
                   ax=None, 
                   figsize=(12, 12), 
                   bgcolor='Black', 
                   node_color='y', 
                   node_size=5,
                   node_zorder=1, 
                   edge_color='w', 
                   edge_linewidth=0.5)

In [ ]:
# Setting up the starting and ending point lat/lng
orig_node = ox.distance.nearest_nodes(G, X=114.2255 , Y=22.3135)
dest_node = ox.distance.nearest_nodes(G, X=114.1717, Y=22.2950)

# Computing the shortest path by length
route = nx.shortest_path(G, orig_node, dest_node, weight="length")


fig, ax = ox.plot_graph_route(
    G,
    route,
    route_linewidth=2.5,  
    route_color="r",  
    node_size=5,  
    node_color="y",  
    edge_color="w",  
    edge_linewidth=0.5,  
    bgcolor="black", 
    figsize=(12, 12),
)
plt.show()

In [ ]:
# Computing the degree centrality for the nodes
degree_centrality_nodes = nx.degree_centrality(G)

In [ ]:
# Computing the betweenness centrality for the nodes
betweenness_centrality_nodes = nx.betweenness_centrality(G)

In [ ]:
import osmnx as ox
from IPython.display import Image
warnings.filterwarnings('ignore')

#Setting the configuration of the image
img_folder = "images"
extension = "png"
size = 400
dpi = 1080


# Customizing the size of the street
street_widths = {
    "footway": 0.5,
    "steps": 0.5,
    "pedestrian": 0.5,
    "path": 0.5,
    "track": 0.5,
    "service": 2,
    "residential": 3,
    "primary": 5,
    "motorway": 6,
}

# Setting up central point
point = (22.2950, 114.1717)
fp = f"./{img_folder}/{place}.{extension}"

# Customizing the color
bgcolor = "white"      
street_color = "Black"   

# plotting the figure ground
fig, ax = ox.plot_figure_ground(
    point=point,
    filepath=fp,
    network_type="all",
    street_widths=street_widths,
    dpi=dpi,
    save=True,
    show=False,
    close=True,
    bgcolor=bgcolor,
    color=street_color,
)
Image(fp, height=size, width=size)

In [ ]:
# Interactive maps to plot edges
ox.graph_to_gdfs(G, nodes=False).explore()

In [ ]:
# Interactive maps to plot nodes
nodes = ox.graph_to_gdfs(G, edges=False)
nodes.explore(tiles="cartodbpositron", marker_kwds={"radius": 5})

In [ ]:
# Interactive maps to plot nodes+edges

nodes, edges = ox.graph_to_gdfs(G)
m = edges.explore(color="blue", tiles="cartodbdarkmatter")
nodes.explore(m=m, color="yellow", marker_kwds={"radius": 5})

In [ ]:
# interactive maps for centrality measures - betweenness_centrality

nx.set_node_attributes(G, betweenness_centrality, name="bc")
nodes = ox.graph_to_gdfs(G, edges=False)
nodes.explore(tiles="cartodbdarkmatter", column="bc", marker_kwds={"radius": 5})

In [ ]:
# Export the street network to a GeoPackage (.gpkg) file.
ox.save_graph_geopackage(G, filepath="/UA/map1.gpkg") # designated path to save

In [ ]:
import folium
import geopandas as gpd

# Reading the saved .gpkg file with gpd and plot it with folium map
gpkg_filepath = "/UA/map1.gpkg"
nodes = gpd.read_file(gpkg_filepath, layer="nodes") # Assigning nodes and edges
edges = gpd.read_file(gpkg_filepath, layer="edges")

# Creating the base map
map_center = [22.2950, 114.1717]
m = folium.Map(location=map_center, zoom_start=15)


folium.GeoJson(edges.to_json(), name="Edges").add_to(m)
folium.GeoJson(nodes.to_json(), name="Nodes").add_to(m)
folium.LayerControl().add_to(m)

m

In [ ]:
# Extracting building features

warnings.filterwarnings('ignore')
buildings = ox.features_from_place(place, tags={'building': True})

In [ ]:
# Extracting parks features
warnings.filterwarnings('ignore')
parks = ox.features_from_place(place, tags={'leisure': 'park'})

In [ ]:
# Plotting the extracted features

fig, ax = plt.subplots(figsize=(12, 12))

buildings.plot(ax=ax, facecolor='lightgrey', edgecolor='black', alpha=0.7, linewidth=0.5)
parks.plot(ax=ax, facecolor='green', edgecolor='none', alpha=0.5)

ax.set_title("Buildings and Parks", fontsize=16)
plt.axis("off")
plt.show()    